In [1]:
import os, re
import pandas as pd
import numpy as np
import sqlalchemy as sa
from sqlalchemy.types import Integer, Float, Date, VARCHAR
from datetime import datetime

### Загрузка результатов краулера в формате для связки со спарк (учитываются только инн и огрн)

#### Сбор справочника мобильных кодов

In [2]:
mobile_codes = pd.read_csv('~/parsers/parsers_resources/common/mobiles_codes.csv', sep= ';', encoding= 'cp1251')
mobile_codes = mobile_codes['АВС/ DEF'].drop_duplicates().astype(str).values

In [3]:
def group_values(values, max_len):
    values = [value for value in values if pd.notnull(value)]
    
    res = dict()
    for value in values:
        tmp_values = value.split(', ')
        tmp_values = [x for x in tmp_values if len(x) > 0]
        for x in tmp_values:
            if x in res:
                res[x] += 1
            else:
                res[x] = 1
    
    res = sorted(res.items(), key= lambda x : x[1], reverse= True)
    res = [x[0] for x in res]
    res_len = 0
    fin_res = list()
    for value in res:
        res_len += len(value) + 2
        if res_len < max_len:
            fin_res.append(value)
        else:
            break
    
    if len(fin_res) > 0:
        fin_res = ', '.join(fin_res)
    else:
        fin_res = np.nan
    
    return fin_res

In [4]:
def get_mob_code(norm_tel):
    return norm_tel[:3]

In [5]:
def is_mobile_num(norm_tel):
    if get_mob_code(norm_tel) in mobile_codes:
        return True
    else:
        return False

In [6]:
def group_telnumbers(values, max_len):
    values = [value for value in values if pd.notnull(value)]
    
    res = dict()
    for value in values:
        phones = value.split(', ')
        phones = [x for x in phones if len(x) > 0]
        for phone in phones:
            norm_phone = re.sub(r'\D', '', phone)
            if len(norm_phone) == 11:
                norm_phone = norm_phone[1:]

            if norm_phone in res:
                res[norm_phone] += 1
            else:
                res[norm_phone] = 1
    
    res = sorted(res.items(), key= lambda x : (is_mobile_num(x[0]), x[1]), reverse= True)
    res = [x[0] for x in res]
    res_len = 0
    fin_res = list()
    for value in res:
        res_len += len(value) + 2
        if res_len < max_len:
            fin_res.append(value)
        else:
            break
            
    if len(fin_res) > 0:
        fin_res = ', '.join(fin_res)
    else:
        fin_res = np.nan
        
    return fin_res

In [9]:
def get_contacts(*filenames):
    csv_col_names = ['website', 'webpage', 'names', 'inns', 'ogrns', 'city', 'telnumbers', 'emails']
    contacts = pd.DataFrame(columns=csv_col_names)
    for file in filenames:
        tmp_contacts = pd.read_csv(file, sep= ';', quotechar = '\"', quoting=1, header= None, names= csv_col_names,\
                               encoding= 'utf-8')
        contacts = contacts.append(tmp_contacts, ignore_index=True)
        
    contacts = contacts.drop(columns=['names', 'city',])
    contacts = contacts.drop_duplicates()
    contacts.inns = contacts.inns.str.replace('ИНН: ', '')
    contacts.ogrns = contacts.ogrns.str.replace('ОГРН: ', '')
    contacts.telnumbers = contacts.telnumbers.str.replace('Тел.:', '')
    contacts.emails = contacts.emails.str.replace('Почта: ', '')
    contacts = contacts[(contacts.inns != '') | (contacts.ogrns != '')]
    
    flat_inns = contacts[['webpage', 'inns']]
    flat_inns = flat_inns.set_index('webpage')
    flat_inns = flat_inns.inns.str.split(', ').apply(pd.Series).unstack().dropna().reset_index(0, drop=True)
    flat_inns.name = 'inn'
    flat_inns = flat_inns[flat_inns != '']
    
    flat_ogrns = contacts[['webpage', 'ogrns']]
    flat_ogrns = flat_ogrns.set_index('webpage')
    flat_ogrns = flat_ogrns.ogrns.str.split(', ').apply(pd.Series).unstack().dropna().reset_index(0, drop=True)
    flat_ogrns.name = 'ogrn'
    flat_ogrns = flat_ogrns[flat_ogrns != '']
    
    contacts = contacts.drop(columns=['inns', 'ogrns',])
    contacts = contacts.merge(flat_inns, how='left', left_on='webpage', right_index= True)
    contacts = contacts.merge(flat_ogrns, how='left', left_on='webpage', right_index= True)
    
    return contacts

In [12]:
filenames = ['./res/rf_inn_contacts.txt' , './res/su_inn_contacts.txt']
contacts = get_contacts(*filenames)

In [10]:
filename = '~/parsers/parsers_resources/contacts_scraper/res/su_inn_contacts.txt'
contacts = contacts.append(get_contacts(filename), ignore_index=True)

In [11]:
filename = '~/parsers/parsers_resources/contacts_scraper/res/rf_inn_contacts.txt'
contacts = contacts.append(get_contacts(filename), ignore_index=True)

In [12]:
# filename = './crawler/res/com_fast_inn_contacts.txt'
# contacts = contacts.append(get_contacts(filename), ignore_index=True)

In [12]:
contacts[['inn', 'ogrn']] = contacts[['inn', 'ogrn']].fillna(value='')

In [13]:
contacts = contacts.groupby(by=['inn', 'ogrn'], as_index=False).agg({
                                            'website' : lambda x : group_values(x, 1500),
                                            'telnumbers' : lambda x : group_telnumbers(x, 600),
                                            'emails' : lambda x : group_values(x, 700),
                                            })

In [14]:
contacts = contacts.rename({'website' : 'websites'}, axis='columns')

In [15]:
def writesql_contacts_res(contacts, auth_data, tablename):
    login, password, tns = auth_data[0], auth_data[1], auth_data[2]
    conn_str = 'oracle+cx_oracle://' + login + ':' + password + '@' + tns
    oracle_db = sa.create_engine(conn_str, encoding= 'utf-8', max_identifier_length=128)
    
    metadata = sa.MetaData(bind=oracle_db)

    table = sa.Table(tablename, metadata,
            sa.Column('inn', VARCHAR(14)),
            sa.Column('ogrn', VARCHAR(15)),
            sa.Column('websites', VARCHAR(1700)),
            sa.Column('telnumbers', VARCHAR(750)),
            sa.Column('emails', VARCHAR(800)),
            oracle_compress=True)
    table.drop(checkfirst=True)
    table.create(checkfirst=True)

    table_types = {
        'inn' : VARCHAR(14),
        'ogrn' : VARCHAR(15),
        'websites' : VARCHAR(1700),
        'telnumbers' : VARCHAR(750),
        'emails' : VARCHAR(800),
    }

    connection = oracle_db.connect()
    contacts.to_sql(tablename, con= connection, if_exists= 'append', index= False, chunksize= 500, dtype= table_types) 
    connection.close()

In [18]:
datalab_auth = ('ZASPA_AY[PxU_DCBUL_UMA_MOD]', 'pS_1G1!1', 'DATALAB')

In [19]:
writesql_contacts_res(contacts, datalab_auth, 'zay_crawler_contacts')